## Welcome to the advanced version of the UMARV 2024-2025 PyTorch coding checkpoint

In this notebook you will be creating a basic CNN architecture that performormes image recognition. This will give you the necessary experience you need to get started designing models for the robot. 

# This should take around 1-3 hours
You may look to Ryan or online resources for help if you want to meet this time. You are obviously welcome to go beyond this time to get more out of the checkpoint

# Step One 
Please download the dataset 

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

N-out = ((N-in + 2p - k)/s)+1

In [7]:
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        '''
        Network
        conv -> ReLU -> max pool -> conv -> ReLU -> max pool -> fully connected
        '''
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=1, padding=1) #same padding
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)) #shrinks the image to half its size
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=1, padding=1) #same padding
        self.fc1 = nn.Linear(16*7*7, num_classes) #channels * feature size
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        
        return x
    

        

In [8]:
model = CNN()
x = torch.randn(64, 1,28,28)
print(model(x).shape)

torch.Size([64, 10])


# Now lets load the data

In [10]:
batch_size = 64
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True) #To Tensor for effeciency
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_channel = 1
num_classes = 10 # numbers 0-9
learning_rate = 0.001

num_epochs = 10

model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device) # send to GPU
        targets = targets.to(device=device)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"Completed {epoch} epochs so far")
        


Completed 0 epochs so far
Completed 1 epochs so far
Completed 2 epochs so far
Completed 3 epochs so far
Completed 4 epochs so far
Completed 5 epochs so far
Completed 6 epochs so far
Completed 7 epochs so far
Completed 8 epochs so far
Completed 9 epochs so far


In [24]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("checking accuracy on training data")
    else:
        print("checking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
    model.train()
    
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

checking accuracy on training data
Got 58026 / 60000 with accuracy 96.71
checking accuracy on test data
Got 9701 / 10000 with accuracy 97.01
